In [5]:
import pandas as pd
import numpy as np
import os

In [7]:
# Load the DataFrame from the pickle file
ords_prods_merge = pd.read_pickle(r"C:\Users\gkgiz\Desktop\CF\Achievement 4\09 - 2024 Instacart Basket Analysis\02 Data\Prepared Data\updated_orders.pkl")

MemoryError: 

In [ ]:
df = ords_prods_merge

In [12]:
df.shape

(32434212, 21)

In [13]:
df.groupby('department_id').agg({'order_number': ['mean']})
#Step 2: Find the average order number per department id for the entire dataset

,order_number
,mean
department_id,
1,15.457687
2,17.277920
3,17.179756
4,17.811403
5,15.213779
6,16.439806
7,17.225773
8,15.340520


Step 3: Analyze the result. How do the results for the entire dataframe differ from those of the subset? Include your comments in a markdown cell below the executed code.
The departments with the highest average of order number are the same in both scenarios (20, 10), however the lowest department differs, as in it is number 1 in the subset of the dataframe, while it is number 5 in the entire dataframe.


In [15]:
# Step 4
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

C:\Users\gkgiz\AppData\Local\Temp\ipykernel_2516\1239913108.py:2: FutureWarning: The provided callable <function max at 0x0000021431510D60> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)


In [16]:
ords_prods_merge.head()

,Unnamed: 0,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,new_order,product_id,...,_merge,product_name,aisle_id,department_id,prices,merge_indicator,busiest day,busiest days,busiest_period_of_day,max_order
0,0,2539329,1,prior,1,2,8,NaN,True,196,...,both,Soda,77,7,9.0,both,Regularly busy,Regularly busy,Average orders,10
1,0,2539329,1,prior,1,2,8,NaN,True,14084,...,both,Organic Unsweetened Vanilla Almond Milk,91,16,12.5,both,Regularly busy,Regularly busy,Average orders,10
2,0,2539329,1,prior,1,2,8,NaN,True,12427,...,both,Original Beef Jerky,23,19,4.4,both,Regularly busy,Regularly busy,Average orders,10
3,0,2539329,1,prior,1,2,8,NaN,True,26088,...,both,Aged White Cheddar Popcorn,23,19,4.7,both,Regularly busy,Regularly busy,Average orders,10
4,0,2539329,1,prior,1,2,8,NaN,True,26405,...,both,XL Pick-A-Size Paper Towel Rolls,54,17,1.0,both,Regularly busy,Regularly busy,Average orders,10


In [17]:
#Creating a loyalty flag
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [18]:
df['max_order']


0           10
1           10
2           10
3           10
4           10
            ..
32434207    13
32434208    13
32434209    13
32434210    13
32434211    13
Name: max_order, Length: 32434212, dtype: int64

In [19]:
price_stats = ords_prods_merge.groupby('loyalty_flag')['prices'].agg(['mean', 'median', 'std', 'min', 'max'])

In [20]:
price_stats

,mean,median,std,min,max
loyalty_flag,,,,,
Loyal customer,10.388747,7.4,327.864108,1.0,99999.0
New customer,13.294370,7.4,597.301692,1.0,99999.0
Regular customer,12.496203,7.4,539.478009,1.0,99999.0


Step 5: The mean of the new customers is a bit higher, but it is not a drastic difference. It could mean that new customers are buying more expensive stuff compared to the rest of the customer base. Standard deviation is quite high in all situations, and we can say that all customers are buying a mix of low and high-end products. The max being 99999.0 across the table is probably either an error, or an outlier, it needs to be investigated.

In [22]:
ords_prods_merge['average_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

C:\Users\gkgiz\AppData\Local\Temp\ipykernel_2516\1521181374.py:1: FutureWarning: The provided callable <function mean at 0x0000021431511760> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ords_prods_merge['average_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)


In [23]:
#Creating a spending flag
ords_prods_merge.loc[ords_prods_merge['average_price']  <10, 'spending_flag'] = 'Low Spender'
ords_prods_merge.loc[ords_prods_merge['average_price']  >= 10, 'spending_flag'] = 'High Spender'


In [24]:
df['spending_flag']


0           Low Spender
1           Low Spender
2           Low Spender
3           Low Spender
4           Low Spender
               ...     
32434207    Low Spender
32434208    Low Spender
32434209    Low Spender
32434210    Low Spender
32434211    Low Spender
Name: spending_flag, Length: 32434212, dtype: object

In [3]:
ords_prods_merge['order_frequency'] = ords_prods_merge.groupby('user_id')['days_since_prior_order'].transform('median')

NameError: name 'ords_prods_merge' is not defined

In [ ]:
#Creating an order frequency flag
ords_prods_merge.loc[ords_prods_merge['order_frequency']  >20, 'frequency_flag'] = 'Non-Frequent Customer'
ords_prods_merge.loc[(ords_prods_merge['order_frequency'] > 10) & (ords_prods_merge['order_frequency'] <= 20), 'frequency_flag'] = 'Regular Customer'
ords_prods_merge.loc[ords_prods_merge['order_frequency']  <=10, 'frequency_flag'] = 'Frequent Customer'